In [65]:

import warnings
warnings.filterwarnings('ignore')
import pickle
import numpy as np
import pandas as pd
import json
from textblob import TextBlob
import nltk
from scipy import spatial
import torch
import spacy
en_nlp = spacy.load('en')

In [66]:
import wikipedia
print(wikipedia.summary("Wikipedia"))

Wikipedia ( (listen),  (listen) WIK-ih-PEE-dee-ə) is a multilingual, web-based, free encyclopedia based on a model of openly editable and viewable content, a wiki. It is the largest and most popular general reference work on the World Wide Web, and is one of the most popular websites by Alexa rank. It is owned and supported by the Wikimedia Foundation, a non-profit organization which operates on money it receives from donors.Wikipedia was launched on January 15, 2001, by Jimmy Wales and Larry Sanger. Sanger coined its name, as a portmanteau of wiki and "encyclopedia". Initially an English-language encyclopedia, versions in other languages were quickly developed. With 5,756,837 articles, the English Wikipedia is the largest of the more than 290 Wikipedia encyclopedias. Overall, Wikipedia comprises more than 40 million articles in 301 different languages and by February 2014 it had reached 18 billion page views and nearly 500 million unique visitors per month.In 2005, Nature published a 

## Try to create embedding using google universal encoder (Not done yet)

## Convert Json to Pandas Dataframe

In [67]:
questions = []

In [68]:
questions.append("How many Nobel laureates has Oxford educated?")

In [69]:
questions.append("When is Barack Obama born ?")

In [70]:
df2 = pd.DataFrame(data = questions, columns=['question'])

In [71]:
df2['search'] = ""

In [72]:
df2['wiki_context'] = ""

In [73]:
def getWikiContext(df):
#     i = 0
#     for q in df['question'][500:501]:
#         if i % 10 == 0:
    for i in range(0,df.shape[0]):
        print(i)
        q = df['question'][i]
        text = nltk.word_tokenize(q)
        tags = nltk.pos_tag(text)
        newquestion = "";
        for tag in tags:
            if tag[1] in noun_tags:
                newquestion += tag[0] + " "
        if newquestion == "":
            newquestion = q
#         print(newquestion)
        search = wikipedia.search(q)
        print(q)
        df['search'][i] = q
        df['wiki_context'][i] = ""
        for res in search:
            print(res)
#             if df['wiki_context'][i] is None:
            try:
                print(wikipedia.summary(res))
                df['wiki_context'][i] += (wikipedia.summary(res))
#                     print(df['wiki_context'][i])
            except wikipedia.exceptions.DisambiguationError as e:
                continue
#             else:
#                 break
        if df['wiki_context'][i] is None:
            df['wiki_context'][i] = "No wikipedia search result found"
        i = i + 1
    return df

In [74]:
noun_tags = ['NN','NNP','NNS','NNPS']

In [75]:
getWikiContext(df2)

0
How many Nobel laureates has Oxford educated?
James Meade
James Edward Meade,  (23 June 1907 – 22 December 1995) was a British economist and winner of the 1977 Nobel Memorial Prize in Economic Sciences jointly with the Swedish economist Bertil Ohlin for their "pathbreaking contribution to the theory of international trade and international capital movements."
Meade was born in Swanage, Dorset. He was educated at Malvern College and attended Oriel College, Oxford in 1926 to read Greats, but switched to Philosophy, Politics and Economics and gained an outstanding first. His interest in economics grew from an influential postgraduate year at Christ's College, Cambridge and Trinity College, Cambridge (1930–31), where he held frequent discussions with leading economists of the time including Dennis Robertson and John Maynard Keynes.
After working in the League of Nations and the Cabinet Office, he was the leading economist of the early years of Attlee's government, before taking professor

,question,search,wiki_context
0,How many Nobel laureates has Oxford educated?,How many Nobel laureates has Oxford educated?,"James Edward Meade, (23 June 1907 – 22 Decemb..."
1,When is Barack Obama born ?,When is Barack Obama born ?,Barack Hussein Obama Sr. (; 18 June 1936 – 24 ...


In [76]:
predicted['wiki_context'][1]

'The family of Barack Obama, the 44th President of the United States, and his wife Michelle Obama is made up of people of Kenyan (Luo), African-American, and Old Stock American (including originally English, Scots-Irish, Welsh, German, and Swiss) ancestry. Their immediate family was the First Family of the United States from 2009 to 2017. The Obamas are the first First Family of African-American descent.\n\n\n== Immediate family ==\n\n\n=== Michelle Obama ===\n\nMichelle LaVaughn Robinson Obama (born January 17, 1964) is an American lawyer, university administrator, and writer who served as the First Lady of the United States from 2009 to 2017. She is Barack Obama\'s wife, and was the first African-American First Lady. Raised on the South Side of Chicago, Michelle Obama is a graduate of Princeton University and Harvard Law School, and spent her early legal career working at the law firm Sidley Austin, where she met her husband. She subsequently worked as the Associate Dean of Student S

## Create dictionary of sentence embeddings for faster computation

In [77]:
# number of paragraphs
paras = list(df2["wiki_context"].drop_duplicates().reset_index(drop= True))

In [78]:
paras = ['None' if v is None else v for v in paras]

In [79]:
len(paras)

2

In [80]:
# use blob to separate sentence
blob = TextBlob(" ".join(paras))
sentences = [item.raw for item in blob.sentences]

In [81]:
len(sentences)

192

In [82]:
# use infersent to encode sentence
infersent = torch.load('InferSent/encoder/infersent.allnli.pickle', map_location=lambda storage, loc: storage)
infersent.set_glove_path("InferSent/dataset/GloVe/glove.840B.300d.txt")

In [83]:
infersent.build_vocab(sentences, tokenize=True)

Found 1728(/1797) words with glove vectors
Vocab size : 1728


In [84]:
# create sentence embedding
dict_embeddings = {}
for i in range(0,len(sentences)):
    if (i % 100 == 0):
        print(i)
    dict_embeddings[sentences[i]] = infersent.encode([sentences[i]], tokenize=True)

0
100


In [85]:
questions = list(df2["question"])

In [86]:


len(questions)


2

In [87]:
# create question embedding
for i in range(len(questions)):
    if (i % 100 == 0):
        print(i)
    dict_embeddings[questions[i]] = infersent.encode([questions[i]], tokenize=True)

0


In [88]:
for i, key in enumerate(dict_embeddings):
    print(i)
    print("key is " + key)

0
key is James Edward Meade,  (23 June 1907 – 22 December 1995) was a British economist and winner of the 1977 Nobel Memorial Prize in Economic Sciences jointly with the Swedish economist Bertil Ohlin for their "pathbreaking contribution to the theory of international trade and international capital movements."
1
key is Meade was born in Swanage, Dorset.
2
key is He was educated at Malvern College and attended Oriel College, Oxford in 1926 to read Greats, but switched to Philosophy, Politics and Economics and gained an outstanding first.
3
key is His interest in economics grew from an influential postgraduate year at Christ's College, Cambridge and Trinity College, Cambridge (1930–31), where he held frequent discussions with leading economists of the time including Dennis Robertson and John Maynard Keynes.
4
key is After working in the League of Nations and the Cabinet Office, he was the leading economist of the early years of Attlee's government, before taking professorships at LSE (1

In [89]:
dict_emb = dict_embeddings

In [90]:
def process_data(train):
    
    print("step 1")
#     train['sentences'] = train['context'].apply(lambda x: [item.raw for item in TextBlob(x).sentences])
    train['wiki_sentences'] = train['wiki_context'].apply(lambda x: [item.raw for item in TextBlob(x).sentences])
    
#     print("step 2")
#     train["target"] = train.apply(get_target, axis = 1)
    
    print("step 3")
    train['wiki_sent_emb'] = train['wiki_sentences'].apply(lambda x: [dict_emb[item][0] if item in\
                                                           dict_emb else np.zeros(4096) for item in x])
    print("step 4")
    train['quest_emb'] = train['question'].apply(lambda x: dict_emb[x] if x in dict_emb else np.zeros(4096) )
        
    return train   

In [91]:
train = process_data(df2)

step 1
step 3
step 4


In [92]:
train['wiki_context'][1]

'Barack Hussein Obama Sr. (; 18 June 1936 – 24 November 1982) was a Kenyan senior governmental economist and the father of Barack Obama, the 44th President of the United States. He is a central figure of his son\'s memoir, Dreams from My Father (1995). Obama married in 1954 and had two children with his first wife, Kezia. He was selected for a special program to attend college in the United States and studied at the University of Hawaii. There, Obama met Stanley Ann Dunham, whom he married in 1961, and with whom he had a son, Barack II. She divorced him three years later. The elder Obama later went to Harvard University for graduate school, where he earned an M.A. in economics, and returned to Kenya in 1964. He saw his son Barack once more, when he was about ten.\nIn late 1964, Obama Sr. married Ruth Beatrice Baker, a Jewish-American woman whom he had met in Massachusetts. They had two sons together before separating in 1971 and divorcing in 1973. Obama first worked for an oil company,

In [93]:
def cosine_sim_wiki(x):
    li = []
    for item in x['wiki_sent_emb']:
        li.append(spatial.distance.cosine(item,x["quest_emb"][0]))
    return li   

In [94]:
def pred_idx(distances):
    return np.argmin(distances)   

def get_sent_by_idx(x):
    return x['wiki_sentences'][x["pred_idx_cos"]]

def predictions(train):
    
    train["cosine_sim_wiki"] = train.apply(cosine_sim_wiki, axis = 1)

    
    print("cosine start")
    
    train["pred_idx_cos"] = train["cosine_sim_wiki"].apply(lambda x: pred_idx(x))

    train["pred_sen_cos"] = train.apply(get_sent_by_idx,axis = 1)


    return train

In [95]:
predicted = predictions(train)

cosine start


In [98]:
predicted

,question,search,wiki_context,wiki_sentences,wiki_sent_emb,quest_emb,cosine_sim_wiki,pred_idx_cos,pred_sen_cos
0,How many Nobel laureates has Oxford educated?,How many Nobel laureates has Oxford educated?,"James Edward Meade, (23 June 1907 – 22 Decemb...","[James Edward Meade, (23 June 1907 – 22 Decem...","[[0.060178213, 0.14402713, 0.10489873, 0.09181...","[[0.08558939, 0.06849944, 0.06744689, 0.020278...","[0.406599760055542, 0.5233998894691467, 0.3752...",16,The university is consistently cited as among ...
1,When is Barack Obama born ?,When is Barack Obama born ?,Barack Hussein Obama Sr. (; 18 June 1936 – 24 ...,[Barack Hussein Obama Sr. (; 18 June 1936 – 24...,"[[0.10311202, 0.097919255, 0.13796782, 0.03961...","[[0.13599062, 0.10439181, 0.08953286, 0.010812...","[0.3906702399253845, 0.5466042459011078, 0.420...",78,Obama was succeeded by Republican Donald Trump...


In [100]:
predicted['pred_sen_cos'][1]

'Obama was succeeded by Republican Donald Trump, who won the 2016 presidential election.'